In [ ]:
import dask.dataframe as dd
import pandas as pd
from dask.diagnostics import ProgressBar
import os
import time
import shutil

def process_large_dataset(input_csv, output_csv, target_ddos=2000000, chunk_size=200000):

    print("Bắt đầu xử lý dataset cực lớn...")
    start_time = time.time()
    
    # Tạo thư mục tạm với timestamp để tránh trùng lặp
    temp_dir = f"temp_chunks_{int(time.time())}"
    os.makedirs(temp_dir, exist_ok=True)
    
    try:
        # 1. Đếm tổng số mẫu DDoS
        print("Đang đếm tổng số mẫu DDoS...")
        ddf = dd.read_csv(input_csv)
        total_ddos = ddf[ddf['label'] == 1].shape[0].compute()
        print(f"Tổng số mẫu DDoS: {total_ddos:,}")
        
        # 2. Tính toán tỷ lệ lấy mẫu
        sample_frac = target_ddos / total_ddos
        print(f"Tỷ lệ lấy mẫu: {sample_frac:.4f}")
        
        # 3. Xử lý từng chunk
        print("Bắt đầu xử lý từng chunk...")
        reader = pd.read_csv(input_csv, chunksize=chunk_size)
        
        sampled_ddos = []
        processed_chunks = 0
        first_benign_chunk = True
        
        for chunk in reader:
            # Lọc và lấy mẫu DDoS
            ddos_chunk = chunk[chunk['label'] == 1]
            if not ddos_chunk.empty:
                sampled = ddos_chunk.sample(frac=sample_frac, random_state=42)
                sampled_ddos.append(sampled)
            
            # Ghi Benign chunks
            benign_chunk = chunk[chunk['label'] == 0]
            if not benign_chunk.empty:
                benign_path = os.path.join(temp_dir, "benign.csv")
                benign_chunk.to_csv(
                    benign_path,
                    index=False,
                    mode='a',
                    header=first_benign_chunk
                )
                first_benign_chunk = False
            
            processed_chunks += 1
            if processed_chunks % 10 == 0:
                print(f"Đã xử lý {processed_chunks} chunks...")
        
        # 4. Ghi mẫu DDoS đã lấy
        if sampled_ddos:
            final_ddos = pd.concat(sampled_ddos)
            final_ddos.to_csv(
                os.path.join(temp_dir, "ddos_samples.csv"),
                index=False
            )
        
        # 5. Kết hợp các file tạm
        print("Đang hợp nhất các file tạm...")
        
        # Đọc lại bằng Dask
        ddos_path = os.path.join(temp_dir, "ddos_samples.csv")
        benign_path = os.path.join(temp_dir, "benign.csv")
        
        if os.path.exists(ddos_path) and os.path.exists(benign_path):
            balanced = dd.concat([
                dd.read_csv(ddos_path),
                dd.read_csv(benign_path)
            ])
        elif os.path.exists(ddos_path):
            balanced = dd.read_csv(ddos_path)
        else:
            balanced = dd.read_csv(benign_path)
        
        # 6. Ghi file kết quả
        with ProgressBar():
            balanced.to_csv(
                output_csv,
                index=False,
                single_file=True
            )
        
        # 7. Thống kê
        final_count = balanced.shape[0].compute()
        final_ddos_count = balanced[balanced['label'] == 1].shape[0].compute() if os.path.exists(ddos_path) else 0
        
        print("\n" + "="*50)
        print(f"HOÀN THÀNH! File đã được lưu tại: {output_csv}")
        print(f"Tổng số mẫu: {final_count:,}")
        print(f"- Mẫu DDoS: {final_ddos_count:,} ({(final_ddos_count/final_count)*100:.1f}%)")
        print(f"- Mẫu Benign: {final_count-final_ddos_count:,} ({(1-final_ddos_count/final_count)*100:.1f}%)")
        print(f"Thời gian xử lý: {(time.time()-start_time)/60:.2f} phút")
        print("="*50)
        
    finally:
        # Dọn dẹp thư mục tạm
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)

if __name__ == "__main__":
    process_large_dataset(
        input_csv='2Type_BenDoS_Mapped_Dataset.csv',
        output_csv='balanced_dataset_final.csv',
        target_ddos=2000000,
        chunk_size=200000
    )

Bắt đầu xử lý dataset cực lớn...
Đang đếm tổng số mẫu DDoS...
Tổng số mẫu DDoS: 42,075,298
Tỷ lệ lấy mẫu: 0.0475
Bắt đầu xử lý từng chunk...
Đã xử lý 10 chunks...
Đã xử lý 20 chunks...
Đã xử lý 30 chunks...
Đã xử lý 40 chunks...
Đã xử lý 50 chunks...
Đã xử lý 60 chunks...
Đã xử lý 70 chunks...
Đã xử lý 80 chunks...
Đã xử lý 90 chunks...
Đã xử lý 100 chunks...
Đã xử lý 110 chunks...
Đã xử lý 120 chunks...
Đã xử lý 130 chunks...
Đã xử lý 140 chunks...
Đã xử lý 150 chunks...
Đã xử lý 160 chunks...
Đã xử lý 170 chunks...
Đã xử lý 180 chunks...
Đã xử lý 190 chunks...
Đã xử lý 200 chunks...
Đã xử lý 210 chunks...
Đang hợp nhất các file tạm...
[########################################] | 100% Completed | 38.06 s

HOÀN THÀNH! File đã được lưu tại: balanced_dataset_final.csv
Tổng số mẫu: 3,098,197
- Mẫu DDoS: 2,000,002 (64.6%)
- Mẫu Benign: 1,098,195 (35.4%)
Thời gian xử lý: 3.35 phút
